In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import sys
import os

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from src.data.make_dataset import CRCHistoPhenotypes_Patch

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.noise import GaussianNoise


Using TensorFlow backend.


# Load dataset

In [9]:
dataset = CRCHistoPhenotypes_Patch(patch_size=(27,27))
X, y = dataset.load_patches()

In [10]:
labelencoder = LabelEncoder()
labelencoder.fit(y)
y = labelencoder.transform(y)

In [11]:
len(X)

22444

In [12]:
batch_size = 32
num_classes = 4
epochs = 200
data_augmentation = False
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_CRCPatches_trained_model.h5'

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)
len_X_train = len(X_train)
X_train = np.concatenate(X_train, axis=0).reshape((len_X_train, )+X[0].shape)

len_X_test = len(X_test)
X_test = np.concatenate(X_test, axis=0).reshape((len_X_test, )+X[0].shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Define a Keras Sequential model

In [15]:
model = Sequential()
model.add(GaussianNoise(stddev=0.1, input_shape=(27, 27, 3)))
model.add(Convolution2D(filters=32, kernel_size=(7, 7), activation='relu', input_shape=(27, 27, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Convolution2D(filters=48, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='tanh'))
# model.add(Dropout(0.25))
model.add(Dense(64, activation='tanh'))
# model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))


In [16]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [17]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=4)


Not using data augmentation.
Train on 15037 samples, validate on 7407 samples
Epoch 1/200
15037/15037 [==============================] - 7s - loss: 1.3222 - acc: 0.3700 - val_loss: 1.1268 - val_acc: 0.5449
Epoch 2/200
15037/15037 [==============================] - 3s - loss: 1.0899 - acc: 0.5475 - val_loss: 0.9208 - val_acc: 0.6640
Epoch 3/200
15037/15037 [==============================] - 3s - loss: 0.9219 - acc: 0.6441 - val_loss: 0.9239 - val_acc: 0.6467
Epoch 4/200
15037/15037 [==============================] - 3s - loss: 0.8764 - acc: 0.6627 - val_loss: 0.8313 - val_acc: 0.6784
Epoch 5/200
15037/15037 [==============================] - 3s - loss: 0.8489 - acc: 0.6771 - val_loss: 0.8944 - val_acc: 0.6468
Epoch 6/200
15037/15037 [==============================] - 3s - loss: 0.8246 - acc: 0.6832 - val_loss: 0.7947 - val_acc: 0.6980
Epoch 7/200
15037/15037 [==============================] - 3s - loss: 0.8124 - acc: 0.6869 - val_loss: 0.7632 - val_acc: 0.7027
Epoch 8/200
15037/15037 [=

KeyboardInterrupt: 

In [19]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [20]:
score = model.evaluate(X_test, y_test, verbose=0)
score

[0.72940465018899558, 0.74915620372799774]